In [39]:
import os
import re
import sys
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
from gensim.models.coherencemodel import CoherenceModel
from gensim.test.utils import common_corpus, common_dictionary
import gensim
import preprocessor as p
from preprocessor.api import clean
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [40]:
DATA_DIR = "../Data"
TWEETS_PATH = os.path.join(DATA_DIR, 'tweets')
TREND_PATH = os.path.join(DATA_DIR, 'trends')
SAVE_PATH = os.path.join(DATA_DIR, 'save')
STATS_PATH = os.path.join(DATA_DIR, 'stats')
os.listdir(SAVE_PATH)[:5]

['lda_test_data',
 '2019-07-01_trends.csv',
 'oo-2019-08-30_trends.csv',
 '2019-07-02_trends.csv',
 'lda_train_data']

In [41]:
dfs_train =  pd.read_csv(os.path.join(SAVE_PATH, "lda_train_data"), header=0, parse_dates=['trend_date'])
dfs_test =  pd.read_csv(os.path.join(SAVE_PATH, "lda_test_data"), header=0, parse_dates=['trend_date'])

In [42]:
dfs_train.head()

,author_id,id,text,trend,trend_date
0,3306453554,1146641969067675648,rt muthafuckas in the comic shops talking bou...,zendaya,2019-07-04
1,50921588,1146032549283729409,any nice words for the family of tyler skaggs or,tyler skaggs,2019-07-01
2,948449480549920768,1146512050522247168,rt halle bailey has been casted as ariel to p...,halle bailey,2019-07-03
3,702614012798885888,1146178670429650945,rt no one saw this coming trump nemesis megan...,rapinoe,2019-07-03
4,828134743094931457,1146261214336471040,rt official mv is out now hashtag trend,lightsiscoming,2019-07-02


In [43]:
print(dfs_test.shape)
dfs_test.head()

(5, 5)


,author_id,id,text,trend,trend_date
0,1081429734280507395,1146016388634406912,rt major twist in temple vandalism case after...,chandnichowk,2019-07-02
1,452343139,1145729338865795074,rt kevin durant after leaving the warriors an...,durant,2019-07-01
2,30297312,1145732715251150850,rt miami is finalizing a trade to send center...,hassan,2019-07-01
3,1097771979040407552,1146133296482377728,rt repeat after me jasprit bumrah is the bes...,bumrah,2019-07-02
4,990817480078577664,1145630219073667073,when is kawhi supposed to announce his decision,kawhi,2019-07-01


# LDA

In [44]:
dfsLDA = dfs_train.loc[:,["trend","text"]]
dfsLDA.dropna(inplace=True)
trend_doc = dfsLDA.groupby(['trend'])['text'].apply(lambda x: ','.join(x)).reset_index()
trend_doc

,trend,text
0,acefamily,rt thank you for everything proof that he is ...
1,achilles,kyrie already proved what type of player he is...
2,acuña,rt the honored freddie freeman ronald acua jr...
3,adviceforateenager,with everchanging tax legislation complex lend...
4,aewfyterfest,rt gives a running dvd into the barbed wire ...
...,...,...
1146,여자친구열대야로여름을열때야,"rt sowon dancing to hush infront of yerin,rt ..."
1147,욜라대충트친소,rt music is my life
1148,우리대장윤두준생일축하해,rt happy birthday to highlight dujun we hope ...
1149,위버스,rt weverse is in korean means upper and me...


In [ ]:
# TO CHECK WHAT PERCENTAGE OF THE TWEETS ARE MATCH

# df_ana = pd.read_csv(os.path.join(TWEETS_PATH,"2019-09-01_tweetsevenmorebasic.csv.bz2.bz2"), header=0)
# df_ana.shape
# print(df.shape[0]/df_ana.shape[0])